In [1]:
# Last Update: 26.07.2022 VURAL

from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math
import wfdb

from common import (
    signal_filtering,
    slice_signal,
    annotation_recorder,
)


FILE_NUMBER = 102

RECORD_PATH = f'../data/mit-bih-arrhythmia-database-1.0.0/{FILE_NUMBER}'
ANNOTATION_PATH = f'../data/mit-bih-arrhythmia-database-1.0.0/{FILE_NUMBER}'

# Parameters for slicing the signal
START_MINUTE = 29
END_MINUTE = 30

# First and last sample's index
FIRST_SAMPLE_IDX = 77
LAST_SAMPLE_IDX = 946

# Annotation Name
ANNOTATION_NAME = f'Record_{FILE_NUMBER}_t_{START_MINUTE}-{END_MINUTE}_s_{FIRST_SAMPLE_IDX}-{LAST_SAMPLE_IDX}_TEST'

<h3>Loading the database</h3>

In [2]:
# Loading the record
record_wfdb = wfdb.rdrecord(RECORD_PATH)

record_annotation_wfdb = wfdb.rdann(ANNOTATION_PATH, 'atr')
#wfdb.plot_wfdb(record=record_100_wfdb, title='Record 100')

# Displaying the details of the record 100
#display(record_100_wfdb.__dict__)
#display(record_100_annotation_wfdb.__dict__)

# Converting the record into Data Frame
record = pd.DataFrame(record_wfdb.p_signal, columns=['MLII', 'V1'])

# Converting the annotation into Data Frame
record_annotation_data = {'sample': record_annotation_wfdb.sample, 'symbol': record_annotation_wfdb.symbol}
record_annotation = pd.DataFrame( record_annotation_data, columns=['sample', 'symbol'])

# Converting into numpy array
ecg = np.array(record['MLII'])

ecg_annotation_index = np.array(record_annotation['sample'])

<h3>Setting the parameters</h3>

In [3]:
#Moving Average

time, sliced_signal = slice_signal(ecg, 360, START_MINUTE * 60, END_MINUTE * 60)
ecg_signal, isoelectric_line = signal_filtering(sliced_signal, 360)


arr = ecg_signal
window_size = 9

i = 0
# Initialize an empty list to store the moving averages
moving_averages = []
  
while i < len(arr) - window_size + 1:
    
    # Store elements from i to (i + window_size)
    window = arr[i : i + window_size]
    # Calculate the average of current window
    window_average = round(sum(window) / window_size, 2)
      
    # Store the average of the current window in the moving average list
    moving_averages.append(window_average)
      
    # Shift the window to right
    i += 1

In [5]:
#https://stackoverflow.com/questions/37365357/when-i-use-matplotlib-in-jupyter-notebook-it-always-raise-matplotlib-is-curren
%matplotlib qt 
plt.plot(time[FIRST_SAMPLE_IDX: LAST_SAMPLE_IDX], isoelectric_line[FIRST_SAMPLE_IDX: LAST_SAMPLE_IDX], 'g')
plt.plot(time[FIRST_SAMPLE_IDX: LAST_SAMPLE_IDX], moving_averages[FIRST_SAMPLE_IDX - 4: LAST_SAMPLE_IDX - 4], 'y')
plt.scatter(time[FIRST_SAMPLE_IDX: LAST_SAMPLE_IDX], ecg_signal[FIRST_SAMPLE_IDX: LAST_SAMPLE_IDX], marker='.')

plt.xlabel('Time (s)', fontweight='bold')
plt.ylabel('Amp', fontweight='bold')
plt.title('Manual Annotation\n P_s, P, P_e, Q_s, Q, R, S, S_e, T_s, T, T_e', fontweight ="bold")
    
x = np.array(plt.ginput(11, timeout=-1, mouse_add=None, mouse_pop=None))
print(x)
x = x[:, 0]

annotation_recorder(x, ANNOTATION_NAME, time)

[[ 1.74099455e+03 -2.41260110e-02]
 [ 1.74107626e+03 -2.06012974e-02]
 [ 1.74115797e+03 -4.87990061e-02]
 [ 1.74124449e+03 -5.23237197e-02]
 [ 1.74137186e+03  5.82124726e-01]
 [ 1.74145838e+03 -3.57211445e-01]
 [ 1.74154730e+03 -7.69967148e-02]
 [ 1.74164583e+03 -5.58484333e-02]
 [ 1.74172273e+03  8.69024670e-02]
 [ 1.74179723e+03 -2.97772946e-03]
 [ 1.74192220e+03 -1.00271566e-02]]
   P_s    P  P_e  Q_s    Q    R    S  S_e  T_s    T  T_e
0  358  387  416  448  493  525  557  592  620  647  691
